In [379]:
import json
import copy
import random
import numpy as np
import pandas as pd
import math

In [172]:
lecturers = ['Kartelj', 'Cukic']
classes = ['Racunarske mreze', 'Funkcionalno programiranje']
groups = ['4i', '4r']
rooms = ['704', '718']
time_slots = range(0, 60)
k = 5

In [8]:
lecturers = pd.DataFrame(np.array([['Kartelj'], ['Cukic'], ['Mitic'], ['Malkov']]),
                   columns=['name'])
lecturers

,name
0,Kartelj
1,Cukic
2,Mitic
3,Malkov


In [37]:
classes = pd.DataFrame(np.array([['Racunarske mreze', None, 2], ['Racunarske mreze - vezbe', 'racunari', 3], 
                                 ['Racunarska inteligencija', None, 2], ['Racunarska inteligencija - vezbe', 'racunari', 3], 
                                 ['Funkcionalno programiranje', None, 2], ['Funkcionalno programiranje - vezbe', 'racunari', 3],
                                 ['Programiranje za veb', None, 2], ['Programiranje za veb - vezbe', 'racunari', 3],
                                 ['Istrazivanje podataka 2', None, 2], ['Istrazivanje podataka 2 - vezbe', 'racunari', 3],
                                 ['Razvoj sofvtvera', None, 2], ['Razvoj softvera - vezbe', 'racunari', 3],
                                 ['Projektovanje baza podataka', None, 2], ['Projektovanje baza podataka - vezbe', 'racunari', 3],
                                ]), columns=['name', 'equipment', 'number_of_classes'])
classes

,name,equipment,number_of_classes
0,Racunarske mreze,None,2
1,Racunarske mreze - vezbe,racunari,3
2,Racunarska inteligencija,None,2
3,Racunarska inteligencija - vezbe,racunari,3
4,Funkcionalno programiranje,None,2
5,Funkcionalno programiranje - vezbe,racunari,3
6,Programiranje za veb,None,2
7,Programiranje za veb - vezbe,racunari,3
8,Istrazivanje podataka 2,None,2
9,Istrazivanje podataka 2 - vezbe,racunari,3


In [133]:
groups = pd.DataFrame(np.array([['4i', None], ['4i1a', '4i'], ['4i1b', '4i'], ['4i2a', '4i'], ['4i2b', '4i'],
                                ['4r', None], ['r4gr1', '4r'], ['r4gr2', '4r'], ['r4gr3', '4r']
                               ]), columns=['name', 'parent'])
groups

,name,parent
0,4i,None
1,4i1a,4i
2,4i1b,4i
3,4i2a,4i
4,4i2b,4i
5,4r,None
6,r4gr1,4r
7,r4gr2,4r
8,r4gr3,4r


In [17]:
locations = pd.DataFrame(np.array([['Studentski trg'], ['Sv. Nikole'], ['Jagic']]), columns=['name'])
locations

,name
0,Studentski trg
1,Sv. Nikole
2,Jagic


In [19]:
location_distances = pd.DataFrame(np.array([['0', '1', 1], ['0', '2', 1], ['1', '2', 1]]), columns=['first', 'second', 'hours'])
location_distances

,first,second,hours
0,0,1,1
1,0,2,1
2,1,2,1


In [36]:
var1 = 'Studentski trg'
var2 = 'Jagic'
location_distances[(location_distances['first'] == str(locations[locations['name'] == var1].index[0])) &
                   (location_distances['second'] == str(locations[locations['name'] == var2].index[0]))]

,first,second,hours
1,0,2,1


In [402]:
rooms = pd.DataFrame(np.array([['704', '0', 'racunari'], ['718', '0', 'racunari'], ['706', '0', None], ['821', '0', None],
                               ['BIM', '0', 'racunari'], ['RLAB', '0', 'racunari'], ['830', '0', None], ['N201', '1', 'racunari'],
                               ['N202', '1', 'racunari'], ['N253', '1', None], ['Jag1', '2', 'racunari'],
                               ['Jag2', '2', 'racunari'], ['Jag3', '2', None], ['Jag4', '2', None]
                              ]), columns=['name', 'location', 'equipment'])
rooms

,name,location,equipment
0,704,0,racunari
1,718,0,racunari
2,706,0,None
3,821,0,None
4,BIM,0,racunari
5,RLAB,0,racunari
6,830,0,None


In [30]:
location_distances[location_distances['first'] == '0']

,first,second,hours
0,0,1,1
1,0,2,1


In [56]:
location_distances[location_distances['first'] ==rooms.iloc[0].location]

,first,second,hours
0,0,1,1
1,0,2,1


In [76]:
teaching_curriculum_json = '''{
    "0": {
        "lecturer": "Kartelj",
        "subject": "Racunarkse mreze",
        "group": "4i"
    },
    "1": {
        "lecturer": "Cukic",
        "subject": "Funkcionalno programiranje",
        "group": "4i"
    },
    "2": {
        "lecturer": "Kartelj",
        "subject": "Racunarkse mreze",
        "group": "4r"
    },
    "3": {
        "lecturer": "Cukic",
        "subject": "Funkcionalno programiranje",
        "group": "4r"
    }
}'''

In [97]:
teaching_curriculum = pd.DataFrame(np.array([['0', '0', {'0'}], ['0', '1', {'1', '2'}], ['0', '1', {'3', '4'}], ['3', '10', {'1', '2'}],
                                             ['3', '10', {'3', '4', '5'}], ['3', '11', {'1'}], ['3', '11', {'2'}], ['3', '11', {'3', '5'}],
                                             ['3', '11', {'4'}], ['2', '8', {'0'}], ['2', '9', {'1', '2'}], ['2', '9', {'3', '4'}]                                               
                                            ]), columns=['lecturer', 'class', 'groups'])
teaching_curriculum

,lecturer,class,groups
0,0,0,{0}
1,0,1,"{2, 1}"
2,0,1,"{4, 3}"
3,3,10,"{2, 1}"
4,3,10,"{4, 3, 5}"
5,3,11,{1}
6,3,11,{2}
7,3,11,"{5, 3}"
8,3,11,{4}
9,2,8,{0}


In [121]:
teaching_curriculum_copy = teaching_curriculum.copy()

In [122]:
teaching_curriculum_copy['lecturer'] = lecturers.iloc[teaching_curriculum_copy['lecturer'].tolist()].name.tolist()
teaching_curriculum_copy['class'] = classes.iloc[teaching_curriculum_copy['class'].tolist()].name.tolist()

In [139]:
teaching_curriculum_copy['groups'] = list(map(lambda gs: list(map(lambda g: groups.iloc[int(g)]['name'], gs)), teaching_curriculum_copy['groups'].tolist()))

In [117]:
lecturers.iloc[teaching_curriculum_copy['lecturer'].tolist()].name.tolist()

['Kartelj',
 'Kartelj',
 'Kartelj',
 'Malkov',
 'Malkov',
 'Malkov',
 'Malkov',
 'Malkov',
 'Malkov',
 'Mitic',
 'Mitic',
 'Mitic']

In [140]:
teaching_curriculum_copy

,lecturer,class,groups
0,Kartelj,Racunarske mreze,[4i]
1,Kartelj,Racunarske mreze - vezbe,"[4i1b, 4i1a]"
2,Kartelj,Racunarske mreze - vezbe,"[4i2b, 4i2a]"
3,Malkov,Razvoj sofvtvera,"[4i1b, 4i1a]"
4,Malkov,Razvoj sofvtvera,"[4i2b, 4i2a, 4r]"
5,Malkov,Razvoj softvera - vezbe,[4i1a]
6,Malkov,Razvoj softvera - vezbe,[4i1b]
7,Malkov,Razvoj softvera - vezbe,"[4r, 4i2a]"
8,Malkov,Razvoj softvera - vezbe,[4i2b]
9,Mitic,Istrazivanje podataka 2,[4i]


In [158]:
time_slots = range(0, 60)

In [142]:
timetable = pd.DataFrame(columns = ['room', 'time_slot'])
timetable

,room,time_slot


In [79]:
teaching_curriculum = json.loads(teaching_curriculum_json)

In [403]:
def init_timetable():
    init = []
    for i in range(teaching_curriculum.shape[0]):
        init.append([rooms.iloc[i % len(rooms)].name, time_slots[i // len(rooms)]])
    timetable = pd.DataFrame(np.array(init), columns = ['room', 'time_slot'])
    return timetable

In [325]:
timetable

,room,time_slot
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [113]:
timetable = 
for tc in teaching_curriculum:
    timetable[tc] = None

In [114]:
timetable

{'0': None, '1': None, '2': None, '3': None}

In [82]:
time_slots

range(0, 60)

In [174]:
def init_timetable():
    timetable = {}
    for tc in teaching_curriculum:
        timetable[tc] = None
    for t in timetable:
        i = int(t)
        timetable[t] = (rooms[i % len(rooms)], time_slots[i // len(rooms)])
    return timetable

In [116]:
timetable

{'0': ('704', 0), '1': ('718', 0), '2': ('704', 1), '3': ('718', 1)}

In [404]:
def are_classes_intersected(timetable,teaching_curriculum, i, j):
    first = timetable.iloc[i]['time_slot'] + classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['number_of_classes'] < timetable.iloc[j].time_slot
    second = timetable.iloc[j]['time_slot'] + classes.iloc[int(teaching_curriculum.iloc[j]['class'])]['number_of_classes'] < timetable.iloc[i].time_slot
    return not (first or second)

In [405]:
def are_groups_intersected(teaching_curriculum, i, j):
    set1 = teaching_curriculum.iloc[i]['groups']
    set2 = teaching_curriculum.iloc[j]['groups']
    for el1 in set1:
        for el2 in set2:
            if el1 == el2 or groups.iloc[int(el1)]['name'] == groups.iloc[int(el2)]['parent'] or groups.iloc[int(el1)]['name'] == groups.iloc[int(el2)]['parent']:
                return True
    return False

In [273]:
are_groups_intersected(teaching_curriculum, 0, 4)

True

In [406]:
def lecturer_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        for j in range(i+1, teaching_curriculum.shape[0]):
            if teaching_curriculum.iloc[i]['lecturer'] == teaching_curriculum.iloc[j]['lecturer'] and are_classes_intersected(timetable, teaching_curriculum, i, j):
                count += 1
    return count

In [407]:
lecturer_constraint(timetable,teaching_curriculum)

1

In [408]:
def group_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        for j in range(i+1, teaching_curriculum.shape[0]):
            if are_groups_intersected(teaching_curriculum, i, j) and are_classes_intersected(timetable, teaching_curriculum, i, j):
                count += 1
    return count

In [289]:
group_constraint(timetable, teaching_curriculum)

31

In [409]:
def room_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        for j in range(i+1, teaching_curriculum.shape[0]):
            if timetable.iloc[i]['room'] == timetable.iloc[j]['room'] and are_classes_intersected(timetable, teaching_curriculum, i, j):
                count += 1
    return count

In [401]:
room_constraint(timetable, teaching_curriculum)

0

In [410]:
def calculate_fittnes(timetable, teaching_curriculum):
    return lecturer_constraint(timetable, teaching_curriculum) + group_constraint(timetable, teaching_curriculum) + room_constraint(timetable, teaching_curriculum)

In [293]:
calculate_fittnes(timetable, teaching_curriculum)

52

In [411]:
def local_search(timetable, teaching_curriculum):
    best_timetable = timetable.copy()
    best_fitness = calculate_fittnes(timetable, teaching_curriculum)
    while(True):
        for i in range(timetable.shape[0]):
            timeslot_range = range(12 * (timetable.iloc[i]['time_slot'] // 12), 12 * (timetable.iloc[i]['time_slot'] // 12 + 1) - classes.iloc[int(teaching_curriculum.iloc[i]['class'])]['number_of_classes'] + 1)
            for timeslot in timeslot_range:
                current_timetable = timetable.copy()
                current_timetable.iloc[i]['time_slot'] = timeslot
                current_fittness = calculate_fittnes(current_timetable, teaching_curriculum)
                if current_fittness < best_fitness:
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
            for j in range(rooms.shape[0]):
                current_timetable = timetable.copy()
                current_timetable.iloc[i]['room'] = rooms.iloc[j].name
                current_fittness = calculate_fittnes(current_timetable, teaching_curriculum)
                if current_fittness < best_fitness:
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
        if best_fitness < calculate_fittnes(timetable, teaching_curriculum):
            print(best_fitness)
            timetable = best_timetable.copy()
        else:
            break
    return timetable

In [412]:
def shaking(timetable, teaching_curriculum):
    time_table_shaking = timetable.copy()
    for j in range(k):
        random_timetable_index = random.choice(range(timetable.shape[0]))
        random_room_index = random.choice(range(rooms.shape[0]))
        random_timeslot = random.choice(time_slots)
        while(random_timeslot % 12 + classes.iloc[int(teaching_curriculum.iloc[random_timetable_index]['class'])]['number_of_classes'] > 11):
            random_timeslot = random.choice(time_slots)
        time_table_shaking.iloc[random_timetable_index] = np.array([[random_room_index, random_timeslot]])
    return time_table_shaking

In [413]:
k = 40
result_timetable = init_timetable()
print('init', calculate_fittnes(result_timetable, teaching_curriculum))
for i in range(30):
    print('iteration', i)
    timetable = result_timetable.copy()
    timetable = shaking(timetable, teaching_curriculum)
    timetable = local_search(timetable, teaching_curriculum)
    if calculate_fittnes(timetable, teaching_curriculum) < calculate_fittnes(result_timetable, teaching_curriculum):
        result_timetable = timetable

init 57
iteration 0
3
2
1
0
iteration 1
5
3
2
1
iteration 2
iteration 3
5
3
2
iteration 4
3
2
1
0
iteration 5
2
1


KeyboardInterrupt: 

In [392]:
timeslot_range = range(12 * (timetable.iloc[0]['time_slot'] // 12), 12 * (timetable.iloc[0]['time_slot'] // 12 + 1) - classes.iloc[int(teaching_curriculum.iloc[0]['class'])]['number_of_classes'] + 1)
timeslot_range

range(24, 35)

In [396]:
calculate_fittnes(result_timetable, teaching_curriculum)

0

In [414]:
result_timetable

,room,time_slot
0,0,0
1,4,28
2,4,12
3,0,12
4,0,53
5,4,44
6,4,20
7,2,36
8,5,16
9,4,4


In [398]:
new_timetable = local_search(timetable, teaching_curriculum)

In [363]:
result_timetable

,room,time_slot
0,3,54
1,0,50
2,12,6
3,4,30
4,0,41
5,1,6
6,5,1
7,6,37
8,1,36
9,11,28


In [356]:
k = 5
newtimetable = shaking(timetable, teaching_curriculum)
newtimetable

,room,time_slot
0,0,0
1,2,56
2,2,0
3,2,4
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,6,15


In [323]:
new_timetable = local_search(timetable, teaching_curriculum)
new_timetable

52 1
52 1
52 1
52 1
52 1
52 1
52 1
52 1
52 1
52 1
52 1
52 1


,room,time_slot
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [133]:
def lecturer_constraint(timetable):
    count = 0
    for t1 in timetable:
        for t2 in timetable:
            if t1 != t2:
                if teaching_curriculum[t1]['lecturer'] == teaching_curriculum[t2]['lecturer'] and timetable[t1][1] == timetable[t2][1]:
                    count += 1
    return count

In [134]:
def group_constraint(timetable):
    count = 0
    for t1 in timetable:
        for t2 in timetable:
            if t1 != t2:
                if teaching_curriculum[t1]['group'] == teaching_curriculum[t2]['group'] and timetable[t1][1] == timetable[t2][1]:
                    count += 1
    return count

In [135]:
def room_constraint(timetable):
    count = 0
    for t1 in timetable:
        for t2 in timetable:
            if t1 != t2:
                if timetable[t1][0] == timetable[t2][0] and timetable[t1][1] == timetable[t2][1]:
                    count += 1
    return count

In [136]:
def calculate_fittnes(timetable):
    return lecturer_constraint(timetable) + group_constraint(timetable) + room_constraint(timetable)

In [127]:
calculate_fittnes(timetable)

('704', 0) ('718', 0)
('718', 0) ('704', 0)
('704', 1) ('718', 1)
('718', 1) ('704', 1)


4

In [1]:
def local_search(timetable):
    best_timetable = copy.deepcopy(timetable)
    best_fitness = calculate_fittnes(timetable)
    while(True):
        print('iter')
        for t in timetable:
            if timetable[t][1] % 12 > 0:
                current_timetable = copy.deepcopy(timetable)
                current_timetable[t] = (timetable[t][0], timetable[t][1] - 1)
                current_fittness = calculate_fittnes(current_timetable)
                if current_fittness < best_fitness:
                    best_timetable = copy.deepcopy(current_timetable)
                    best_fitness = current_fittness
            if timetable[t][1] % 12 < 11:
                current_timetable = copy.deepcopy(timetable)
                current_timetable[t] = (timetable[t][0], timetable[t][1] + 1)
                current_fittness = calculate_fittnes(current_timetable)
                if current_fittness < best_fitness:
                    best_timetable = copy.deepcopy(current_timetable)
                    best_fitness = current_fittness
            for room in rooms:
                current_timetable = copy.deepcopy(timetable)
                current_timetable[t] = (room, timetable[t][1])
                current_fittness = calculate_fittnes(current_timetable)
                if current_fittness < best_fitness:
                    best_timetable = copy.deepcopy(current_timetable)
                    best_fitness = current_fittness
        if best_fitness < calculate_fittnes(timetable):
            print(best_fitness)
            timetable = copy.deepcopy(best_timetable)
        else:
            break
    return timetable

In [157]:
timetable

{'0': ('704', 1), '1': ('718', 0), '2': ('704', 2), '3': ('718', 1)}

In [155]:
calculate_fittnes(timetable)

0

In [150]:
best_fitness

6

In [120]:
current_timetable['0'] = ('704', 2)

In [121]:
current_timetable

{'0': ('704', 2), '1': ('718', 0), '2': ('704', 1), '3': ('718', 1)}

In [122]:
timetable

{'0': ('704', 0), '1': ('718', 0), '2': ('704', 1), '3': ('718', 1)}

In [173]:
def shaking(timetable):
    time_table_shaking = copy.deepcopy(timetable)
    for j in range(k):
        keys = list(timetable.keys())
        random_key = keys[random.randint(0, len(keys)-1)]
        time_table_shaking[random_key] = (random.choice(rooms), random.choice(time_slots))
    return time_table_shaking

In [163]:
keys = list(timetable.keys())
random_key = keys[random.randint(0, len(keys)-1)]
print(timetable[random_key])

('704', 2)


In [167]:
random.choice(rooms)

'704'

In [169]:
random.choice(time_slots)

32

In [178]:
result_timetable = init_timetable()
for i in range(10):
    timetable = copy.deepcopy(result_timetable)
    timetable = shaking(timetable)
    timetable = local_search(timetable)
    if calculate_fittnes(timetable) < calculate_fittnes(result_timetable):
        result_timetable = timetable

iter
iter
iter
iter
iter
iter
iter
iter
iter
iter


In [177]:
result_timetable

{'0': ('704', 1), '1': ('718', 0), '2': ('718', 40), '3': ('704', 20)}

In [179]:
result_timetable

{'0': ('704', 26), '1': ('718', 57), '2': ('704', 56), '3': ('718', 1)}